In [ ]:
# Prototype a ResNet
import numpy as np
from nn_models import simple_dnn
from abundances import *
from typing import Dict, List
import glob
import vaex
import tensorflow as tf
import tensorflow_gnn as gnn
from petastorm import make_batch_reader
from petastorm.tf_utils import make_petastorm_dataset
from gcrn.helper_functions import number_densities_from_abundances
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


# Input: abundances, gas density, temperature (10 quantities for chem1)
# Output: equilibrium number densities (8 quantities for chem1)
"""
TODO
  - Write architectures for
    - ResNet
  - Check normal ML algs
    - SVMs
    - Decision Trees/Random Forests
    - XGBoost
  - Functions to standardise input
"""

"""
Pipeline overview:
  - Read in .parquet files
  - Generate dataset (extract density, temperature, EQ number densities)
  - Determine and calculate abundances based on keys
  - Sort input abundance array and output EQ array alphabetically
  - Pass into model with goal:
    - from density, temperature and abundance, map to output EQ number densities
  - Loss function analysis
"""

def load_dataset(directory: str, suffix="*.parquet"):
  files = [f"file://{f}" for f in glob.glob(f"{directory}/{suffix}")]
  print(files)
  with make_batch_reader(files) as reader:
    dataset = make_petastorm_dataset(reader)
    iterator = dataset.make_one_shot_iterator()
    tensor = iterator.get_next()
    with tf.Session() as sess:
      sample = sess.run(tensor)
      print(sample.id)

def load_dataframe(path: str):
  return vaex.open(path)

def minmax(arr):
  return np.nanmin(arr), np.nanmax(arr)

In [ ]:
cemp_dir = "/media/sdeshmukh/Crucial X6/mean_chemistry/combined_cemp"
suffix = "*chem1*.txt"

species = ["H", "H2", "C", "O", "CO", "CH", "OH", "M"]
chem_keys = [f"{s}_EQ" for s in ["H", "H2", "C", "O", "CO", "CH", "OH", "M"]]
input_keys = [*[f"A_{s}" for s in species], "density", "temperature"]
columns = [*chem_keys, "density", "temperature"]
print(columns)
print(input_keys)
test_suffix = "d3t63g40mm30chem1_04*.parquet"  # load one file
test_file = "d3t63g40mm30chem1_087.parquet"
df = load_dataframe(f"{cemp_dir}/{test_file}")
# dataset = load_dataset(cemp_dir, suffix=test_suffix)
# load_dataset(cemp_dir, test_suffix)

In [ ]:
# # Map density and abundances to number densities
# mass_hydrogen = 1.67262171e-24  # g
# abundance_values = np.array([v for v in mm00_abundances.values()])
# percentage_hydrogen = 10**mm00_abundances['H'] / np.sum(10**abundance_values)
# hydrogen_density = df['density'] / (percentage_hydrogen * mass_hydrogen)
# # Index abundance values as species
# abundance_inputs = np.array([mm00_abundances[s] for s in species])
# number_densities = 10**(np.log10(hydrogen_density) + abundance_inputs - 12)

In [ ]:
# Let's just pass the abundances in
abundances = np.array([mm30a04_abundances[s] for s in species])
new_cols = [f"A_{s}" for s in species]

n_elements = df['density'].shape[0]
for i, k in enumerate(new_cols):
  df[k] = np.repeat(abundances[i], n_elements)

In [ ]:
data = df[input_keys]
data["density"] = np.log10(data["density"])
label = df[chem_keys]
for k in chem_keys:
  label[k] = np.log10(label[k])

data_rest, data_test, label_rest, label_test = train_test_split(data.values, label.values, test_size=0.25, random_state=42)
data_train, data_val, label_train, label_val = train_test_split(data_rest, label_rest, test_size=0.1, random_state=42)
print("Train", data_train.shape, label_train.shape)
print("Val", data_val.shape, label_val.shape)
print("Test", data_test.shape, label_test.shape)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(data_train)
data_train = scaler.transform(data_train)
data_test = scaler.transform(data_test)


In [ ]:
for k in chem_keys:
  print(k, np.any(np.isnan(label[k].values)))

In [32]:
import tensorflow.keras as keras
from sklearn.ensemble import RandomForestRegressor

def mlp():
  model = keras.Sequential([
    keras.layers.Dense(32, input_shape=(10,), activation="sigmoid"),
    keras.layers.Dense(32, activation="sigmoid"),
    keras.layers.Dense(32, activation="sigmoid"),
    keras.layers.Dense(8, activation="linear"),
  ])

  return model

def mlp_dropout():
  # Same as mlp() but with Dropout layers between each Dense layer
  model = keras.Sequential([
    keras.layers.Dense(64, input_shape=(10,), activation="sigmoid"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="sigmoid"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="sigmoid"),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(8, activation="linear"),
  ])

  return model

def cnn_1d():
  # 1D CNN
  model = keras.Sequential([
    # # Input
    # keras.layers.Dense(32, input_shape=(10,), activation="sigmoid"),
    # Convolutional Layers
    keras.layers.Conv1D(32, 4, input_shape=(10, 1), activation="sigmoid", padding="same"),
    keras.layers.Conv1D(64, 4, activation="sigmoid"),
    keras.layers.MaxPooling1D(2),
    keras.layers.Flatten(),
    # Output
    keras.layers.Dense(8, activation="linear")
  ])

  return model


def encoder_decoder():
  model = keras.Sequential([
    # Encoder
    keras.layers.Dense(128, input_shape=(10,), activation="sigmoid"),
    keras.layers.Dense(64, activation="sigmoid"),
    keras.layers.Dense(32, activation="sigmoid"),
    # Decoder
    keras.layers.Dense(32, activation="sigmoid"),
    keras.layers.Dense(64, activation="sigmoid"),
    keras.layers.Dense(128, activation="sigmoid"),
    # Output
    keras.layers.Dense(8, activation="linear")
  ])

  return model

def random_forest():
  model = RandomForestRegressor(n_estimators=100, criterion="mae", verbose=1)
  return model

# TensorFlow
# model = mlp()
model = mlp_dropout()
# model = encoder_decoder()
# model = cnn_1d()
model.compile(optimizer="adam", loss="mse", metrics=["mae"])
model.summary()
model.fit(data_train, label_train, batch_size=512, epochs=500,
          validation_data=(data_val, label_val),
          callbacks=keras.callbacks.EarlyStopping(restore_best_weights=True,
                                                  patience=30),
          verbose=1)

# # Sklearn
# model = random_forest()
# model.fit(data_train, label_train)

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 64)                704       
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_15 (Dense)            (None, 64)                4160      
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 8)               

In [33]:
model.evaluate(data_test, label_test)

12176/12176 [==============================] - 26s 2ms/step - loss: 1.3483 - mae: 0.7495


[1.3483366966247559, 0.7495296001434326]

In [42]:
def check_random_idx(model, X_test, y_test, seed=42):
  if seed:
    np.random.seed(seed)
  idx = np.random.randint(0, X_test.shape[0])
  predictions = model.predict(X_test[idx:idx+10])
  truths = y_test[idx:idx+10]

  return predictions, truths

predictions, truths = check_random_idx(model, data_test, label_test, seed=None)
for i, s in enumerate(chem_keys):
  for j in range(1):
    print(f"({s}): Prediction = {predictions[j, i]:.3f}. Truth = {truths[j, i]:.3f}. P - T = {predictions[j, i] - truths[j, i]:.2f}")

1/1 [==============================] - 0s 27ms/step
(H_EQ): Prediction = 16.326. Truth = 15.855. P - T = 0.47
(H2_EQ): Prediction = 13.413. Truth = 15.381. P - T = -1.97
(C_EQ): Prediction = 9.492. Truth = 6.195. P - T = 3.30
(O_EQ): Prediction = 10.418. Truth = 9.770. P - T = 0.65
(CO_EQ): Prediction = 7.044. Truth = 9.488. P - T = -2.44
(CH_EQ): Prediction = 6.360. Truth = 5.083. P - T = 1.28
(OH_EQ): Prediction = 7.893. Truth = 9.679. P - T = -1.79
(M_EQ): Prediction = 15.350. Truth = 15.078. P - T = 0.27
